# Environment Preparation

In [ ]:
!git clone --quiet https://github.com/Az4z3l/CarND-Traffic-Light-Detection.git tfd
  
!cp -r tfd/config config

!git clone --quiet https://github.com/tensorflow/models.git tf_models
  
!cp -r tf_models/research/object_detection object_detection/

!cp -r tf_models/research/slim slim/

!cp tfd/model_main.py object_detection/model_main.py

!rm -rf tfd

!rm -rf tf_models

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

!git clone https://github.com/cocodataset/cocoapi.git

%cd cocoapi/PythonAPI

!make

%cd /content

!cp -r cocoapi/PythonAPI/pycocotools pycocotools/

!rm -rf cocoapi

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content:/content/slim

# Testing Installation

In [ ]:
!python object_detection/builders/model_builder_test.py

# Download Dataset

The ids are the training and evaluation tf record files.

In [ ]:
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

train_file_id = '1S1KDLCS2ZVhJtb-cFUtpWTs5U2sR0oMT'
eval_file_id = '1pwOAFH0CUnXAykS91XsCBI846HJiauqw'

data_dir = 'data'

if not os.path.isdir(data_dir):
  os.makedirs(data_dir)

downloaded = drive.CreateFile({'id': train_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'mixed_train.record'))

downloaded = drive.CreateFile({'id': eval_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'mixed_eval.record'))

# Download Pre-Trained Models

In [ ]:
import urllib.request
import tarfile

MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
MODEL_URL = 'http://download.tensorflow.org/models/object_detection/' + MODEL_FILE

MODELS_DIR = 'models'

if not os.path.isdir(MODELS_DIR):
  os.makedirs(MODELS_DIR)

if not os.path.exists(MODEL_FILE):
  urllib.request.urlretrieve(MODEL_URL, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
os.rename(MODEL_NAME, os.path.join(MODELS_DIR, MODEL_NAME))

# Train the model 

In [ ]:
FINE_TUNED_DIR = os.path.join(MODELS_DIR, 'fine_tuned')

if not os.path.isdir(FINE_TUNED_DIR):
  os.makedirs(FINE_TUNED_DIR)

!python object_detection/model_main.py \
        --pipeline_config_path=config/ssd_mobilenet_v1.config \
        --model_dir=models/fine_tuned/ssd_mobilenet \